In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np

In [7]:
vanguards = sorted([
    "Venom", "Magneto", "Doctor Strange", "The Thing", "Groot", "Hulk", "Thor", "Peni Parker", "Captain America"
])

duelists = sorted([
    "Moon Knight", "Squirrel Girl", "Human Torch", "Black Widow", "Namor", "The Punisher",
    "Hawkeye", "Scarlet Witch", "Psylocke", "Winter Soldier", "Wolverine", "Iron Man", "Hela",
    "Mister Fantastic", "Spider Man", "Iron Fist", "Star Lord", "Black Panther", "Storm", "Magik"
])

strategists = sorted([
    "Jeff The Land Shark", "Luna Snow", "Cloak & Dagger", "Invisible Woman", "Adam Warlock",
    "Loki", "Mantis", "Rocket Raccoon"
])
sorted_heroes = vanguards + duelists + strategists
WinRate_df = pd.DataFrame(index=sorted_heroes)
base_url = "https://rivalsmeta.com/characters/{}/matchups"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

def format_hero_name(hero_name):
    """Converts hero names to the correct URL format."""
    if hero_name == "Cloak & Dagger":
        return "cloak-dagger"  # Special case
    return hero_name.lower().replace(" ", "-")

In [8]:
for hero in sorted_heroes:
    print(f"Fetching matchups for {hero}...")
    hero_url_name = format_hero_name(hero)
    url = base_url.format(hero_url_name)
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to fetch {hero}, status code: {response.status_code}")
        continue
    soup = BeautifulSoup(response.text, "html.parser")
    tables = soup.find_all("tbody")
    matchup_data = {}
    for table in tables:
        for row in table.find_all("tr"):
            columns = row.find_all("td")
            if len(columns) < 3:
                continue 
            opponent_name_tag = columns[0].find("img", class_="hero-img")
            if opponent_name_tag:
                opponent_name = opponent_name_tag["alt"].strip()
            else:
                continue
            win_rate = columns[1].text.strip().replace("%", "")
            if opponent_name in sorted_heroes:  # Ensure the opponent is a valid in-game hero
                matchup_data[opponent_name] = float(win_rate)

    matchup_data[hero] = 50.0
    win_rate_series = pd.Series(matchup_data, name=hero)
    WinRate_df[hero] = win_rate_series
    time.sleep(2)
WinRate_df = WinRate_df.loc[sorted_heroes]

Fetching matchups for Captain America...
Fetching matchups for Doctor Strange...
Fetching matchups for Groot...
Fetching matchups for Hulk...
Fetching matchups for Magneto...
Fetching matchups for Peni Parker...
Fetching matchups for The Thing...
Fetching matchups for Thor...
Fetching matchups for Venom...
Fetching matchups for Black Panther...
Fetching matchups for Black Widow...
Fetching matchups for Hawkeye...
Fetching matchups for Hela...
Fetching matchups for Human Torch...
Fetching matchups for Iron Fist...
Fetching matchups for Iron Man...
Fetching matchups for Magik...
Fetching matchups for Mister Fantastic...
Fetching matchups for Moon Knight...
Fetching matchups for Namor...
Fetching matchups for Psylocke...
Fetching matchups for Scarlet Witch...
Fetching matchups for Spider Man...
Fetching matchups for Squirrel Girl...
Fetching matchups for Star Lord...
Fetching matchups for Storm...
Fetching matchups for The Punisher...
Fetching matchups for Winter Soldier...
Fetching match

In [9]:
WinRate_df.to_csv("MarvelRivals_WinRate_Matrix.csv")
WinRate_df

,Captain America,Doctor Strange,Groot,Hulk,Magneto,Peni Parker,The Thing,Thor,Venom,Black Panther,...,Winter Soldier,Wolverine,Adam Warlock,Cloak & Dagger,Invisible Woman,Jeff The Land Shark,Loki,Luna Snow,Mantis,Rocket Raccoon
Captain America,50.00,47.12,47.87,48.00,46.44,49.81,47.30,49.15,45.90,49.31,...,47.45,47.55,49.65,45.02,45.38,42.70,50.54,44.35,50.54,52.22
Doctor Strange,52.88,50.00,50.56,52.24,49.04,53.25,51.50,52.37,50.13,52.95,...,50.45,51.30,51.93,47.29,49.01,47.00,52.31,48.29,53.14,54.68
Groot,52.13,49.44,50.00,51.31,47.70,53.18,49.94,51.86,49.15,52.75,...,49.23,49.17,50.51,46.74,48.41,46.11,51.23,46.28,52.16,53.07
Hulk,52.00,47.76,48.69,50.00,47.53,51.90,49.20,49.86,49.23,52.57,...,48.79,48.54,50.98,45.51,46.91,43.17,49.65,46.18,52.21,53.62
Magneto,53.56,50.96,52.30,52.47,50.00,54.39,52.25,53.57,51.49,53.58,...,51.72,53.38,52.62,48.27,49.95,47.07,52.64,48.74,54.12,56.21
Peni Parker,50.19,46.75,46.82,48.10,45.61,50.00,47.63,49.68,47.76,53.43,...,47.00,47.11,48.20,44.71,45.74,40.66,48.73,44.17,50.00,52.78
The Thing,52.70,48.50,50.06,50.80,47.75,52.37,50.00,51.37,50.55,51.92,...,49.71,52.91,50.31,46.63,47.60,45.03,50.98,47.02,52.45,54.91
Thor,50.85,47.63,48.14,50.14,46.43,50.32,48.63,50.00,48.12,49.93,...,48.59,47.77,49.55,45.29,46.27,42.43,49.40,45.67,50.92,52.82
Venom,54.10,49.87,50.85,50.77,48.51,52.24,49.45,51.88,50.00,52.78,...,49.84,50.91,52.09,47.73,47.92,45.31,51.08,47.13,53.83,55.94
Black Panther,50.69,47.05,47.25,47.43,46.42,46.57,48.08,50.07,47.22,50.00,...,48.35,51.66,49.54,44.69,45.75,42.21,48.31,44.55,50.41,53.75


In [10]:
NumMatchesdf = pd.DataFrame(index=sorted_heroes)

for hero in sorted_heroes:
    print(f"Fetching match counts for {hero}...")
    hero_url_name = format_hero_name(hero)
    url = base_url.format(hero_url_name)
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to fetch {hero}, status code: {response.status_code}")
        continue
    soup = BeautifulSoup(response.text, "html.parser")
    tables = soup.find_all("tbody")
    match_count_data = {}
    for table in tables:
        for row in table.find_all("tr"):
            columns = row.find_all("td")
            if len(columns) < 3:
                continue 
            opponent_name_tag = columns[0].find("img", class_="hero-img")
            if opponent_name_tag:
                opponent_name = opponent_name_tag["alt"].strip()
            else:
                continue
            match_count_text = columns[2].text.strip().replace(",", "")  # Remove commas
            if match_count_text.isdigit():
                match_count = int(match_count_text)
            else:
                continue  
            if opponent_name in sorted_heroes: 
                match_count_data[opponent_name] = match_count
    if match_count_data:
        match_count_data[hero] = int(np.mean(list(match_count_data.values())))
    match_count_series = pd.Series(match_count_data, name=hero)
    NumMatchesdf[hero] = match_count_series
    time.sleep(2)

NumMatchesdf = NumMatchesdf.loc[sorted_heroes]

Fetching match counts for Captain America...
Fetching match counts for Doctor Strange...
Fetching match counts for Groot...
Fetching match counts for Hulk...
Fetching match counts for Magneto...
Fetching match counts for Peni Parker...
Fetching match counts for The Thing...
Fetching match counts for Thor...
Fetching match counts for Venom...
Fetching match counts for Black Panther...
Fetching match counts for Black Widow...
Fetching match counts for Hawkeye...
Fetching match counts for Hela...
Fetching match counts for Human Torch...
Fetching match counts for Iron Fist...
Fetching match counts for Iron Man...
Fetching match counts for Magik...
Fetching match counts for Mister Fantastic...
Fetching match counts for Moon Knight...
Fetching match counts for Namor...
Fetching match counts for Psylocke...
Fetching match counts for Scarlet Witch...
Fetching match counts for Spider Man...
Fetching match counts for Squirrel Girl...
Fetching match counts for Star Lord...
Fetching match counts f

In [12]:
NumMatchesdf.to_csv("MarvelRivals_NumMatches_Matrix.csv")
NumMatchesdf

54.15